The following predicts the probability of crime occurance on:
1. days in the week
2. hours in the day 
3. Month in the year 
To do this the Random forest model was adopted to categorize and predict the probability of cirme occuring on different days, hours and month.   

In [32]:
#import the necessary libraries 
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, LabelEncoder
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import numpy as np

In [46]:
#import the dataset 
crime_data = pd.read_csv('https://raw.githubusercontent.com/Pelumioluwa/MBAN6110Class/main/Data/Toronto%20Police%20Shooting%20and%20Firearm%20Discharge.csv')
crime_data

,X,Y,OBJECTID,EVENT_UNIQUE_ID,OCC_DATE,OCC_YEAR,OCC_MONTH,OCC_DOW,OCC_DOY,OCC_DAY,...,OCC_TIME_RANGE,DIVISION,DEATH,INJURIES,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
0,-79.234955,43.781528,1,GO-2004397105,2004/05/15 04:00:00+00,2004,May,Saturday,136,15,...,Night,D41,0,0,142,Woburn North,137,Woburn (137),-79.234955,43.781528
1,-79.618218,43.733547,2,GO-2004600109,2004/07/26 04:00:00+00,2004,July,Monday,208,26,...,Evening,D23,0,0,1,West Humber-Clairville,1,West Humber-Clairville (1),-79.618218,43.733547
2,-79.518759,43.769157,3,GO-2004311851,2004/05/14 04:00:00+00,2004,May,Friday,135,14,...,Afternoon,D31,1,1,24,Black Creek,24,Black Creek (24),-79.518759,43.769157
3,-79.378870,43.660665,4,GO-2004736004,2004/12/19 05:00:00+00,2004,December,Sunday,354,19,...,Night,D51,1,0,168,Downtown Yonge East,75,Church-Yonge Corridor (75),-79.378870,43.660665
4,-79.212435,43.812075,5,GO-2004303455,2004/04/12 04:00:00+00,2004,April,Monday,103,12,...,Evening,D42,0,0,146,Malvern East,132,Malvern (132),-79.212435,43.812075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,-79.362941,43.771764,5703,GO-20221038418,2022/06/02 04:00:00+00,2022,June,Thursday,153,2,...,Night,D33,0,1,53,Henry Farm,53,Henry Farm (53),-79.362941,43.771764
5703,-79.402702,43.643920,5704,GO-20221362949,2022/07/17 04:00:00+00,2022,July,Sunday,198,17,...,Night,D14,1,1,164,Wellington Place,77,Waterfront Communities-The Island (77),-79.402702,43.643920
5704,-79.196217,43.786621,5705,GO-2022336423,2022/02/21 05:00:00+00,2022,February,Monday,52,21,...,Night,D43,0,0,135,Morningside,135,Morningside (135),-79.196217,43.786621
5705,-79.443993,43.658083,5706,GO-20221765442,2022/09/11 04:00:00+00,2022,September,Sunday,254,11,...,Night,D11,0,2,83,Dufferin Grove,83,Dufferin Grove (83),-79.443993,43.658083


First step is to predict the probability for crime occurance on the days in the week 
1. Convert day of the week into label 
2. Convert the categorical variable into dummy data and one-encode it 
3. Scale the numerical variables (day of the year,day of the month, hour of the day)
4. Make date time the index



In [47]:
#Separate the different numerical features, categorical features, label and target  
categorical_features = ['OCC_MONTH','OCC_TIME_RANGE']
numerical_features = ['OCC_DOY','OCC_DAY','OCC_HOUR']
target = ['OCC_DOW']
label = ['OCC_DOW']


In [50]:
#define label encoder and convert the days of week into labels
label_encoder_trans = LabelEncoder()
crime_data['OCC_DOW'] = label_encoder_trans.fit_transform(crime_data['OCC_DOW'])


In [51]:
crime_data[target]


,OCC_DOW
0,2
1,1
2,0
3,3
4,1
...,...
5702,4
5703,3
5704,1
5705,3


In [52]:
#Start with day of the week
# split data in train / test
X_train, X_test, y_train, y_test = train_test_split(crime_data[categorical_features + numerical_features]
                                                    ,crime_data[target],
                                                    test_size = 0.3, random_state=1234)

In [53]:
#define transformers and encoder
categorical_transformer = Pipeline(steps=[('onehot',OneHotEncoder(handle_unknown='ignore'))])
numerical_transformer = Pipeline(steps=[('scaler',StandardScaler())])

In [54]:
#build the preprocessor 
preprocessor = ColumnTransformer(transformers=[('categorical',categorical_transformer, categorical_features),
                                               ('numerical',numerical_transformer,numerical_features) ])

In [55]:
#define model
random_forest = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestClassifier())])

In [56]:
#fit the model
random_forest.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['OCC_MONTH',
                                                   'OCC_TIME_RANGE']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['OCC_DOY', 'OCC_DAY',
                                                   'OCC_HOUR'])])),
                ('regressor', RandomForestClassifier())])

In [57]:
# Predict the probabilities of crime occurrence for each day of the week in the test set
day_probabilities = random_forest.predict_proba(X_test)

In [59]:
# Display the probabilities for each day of the week in the test set
day_names = label_encoder_trans.inverse_transform(random_forest.classes_)
probabilities_df = pd.DataFrame(day_probabilities, columns=day_names)
print(probabilities_df)

        Friday    Monday  Saturday    Sunday  Thursday   Tuesday  Wednesday
0     0.357500  0.052500  0.120000  0.110000  0.040000  0.240000   0.080000
1     0.100000  0.010000  0.020000  0.370000  0.000000  0.490000   0.010000
2     0.020000  0.000000  0.171667  0.605000  0.013333  0.108333   0.081667
3     0.513202  0.174798  0.032000  0.090000  0.062000  0.028000   0.100000
4     0.030500  0.277000  0.086000  0.310000  0.082500  0.090000   0.124000
...        ...       ...       ...       ...       ...       ...        ...
1708  0.041119  0.000000  0.218881  0.276476  0.015000  0.408524   0.040000
1709  0.025000  0.420000  0.020000  0.020000  0.252500  0.027500   0.235000
1710  0.423167  0.000000  0.110000  0.436833  0.000000  0.030000   0.000000
1711  0.136000  0.012833  0.150000  0.041167  0.410000  0.000000   0.250000
1712  0.097571  0.043429  0.000000  0.160000  0.000000  0.699000   0.000000

[1713 rows x 7 columns]


In [23]:
#import evaluation metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score


In [68]:
predicted_classes = day_probabilities.argmax(axis=1)
# Inverse transform the numerical class labels back to day names
predicted_day_names = label_encoder_trans.inverse_transform(predicted_classes)

In [69]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print("Accuracy:", accuracy)

Accuracy: 0.20081727962638646


In [70]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[44 31 34 40 37 19 24]
 [33 49 31 46 29 30 22]
 [43 31 73 72 32 25 30]
 [48 34 41 64 45 33 21]
 [31 37 35 34 33 21 19]
 [25 29 36 29 33 47 27]
 [27 35 33 42 23 22 34]]


In [71]:
# Calculate classification report (includes precision, recall, and F1-score for each class)
classification_rep = classification_report(y_test, predicted_classes, target_names=label_encoder_trans.classes_)
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

      Friday       0.18      0.19      0.18       229
      Monday       0.20      0.20      0.20       240
    Saturday       0.26      0.24      0.25       306
      Sunday       0.20      0.22      0.21       286
    Thursday       0.14      0.16      0.15       210
     Tuesday       0.24      0.21      0.22       226
   Wednesday       0.19      0.16      0.17       216

    accuracy                           0.20      1713
   macro avg       0.20      0.20      0.20      1713
weighted avg       0.20      0.20      0.20      1713



In [72]:
# Calculate AUC-ROC (requires one-hot encoded y_test and day_probabilities)
y_test_one_hot = pd.get_dummies(y_test)
auc_roc = roc_auc_score(y_test_one_hot, day_probabilities, multi_class='ovr')
print("AUC-ROC:", auc_roc)

AUC-ROC: 0.5632371833173357


In [73]:
#get the day with the high mean of probability 
probabilities_df.describe(include='all')

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
count,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000
mean,0.146356,0.141178,0.166053,0.176272,0.131699,0.121479,0.116963
std,0.186808,0.193325,0.211212,0.210824,0.183344,0.181809,0.169774
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.013333,0.010000,0.020000,0.020000,0.010000,0.010000,0.010000
50%,0.070000,0.050000,0.070000,0.083333,0.040000,0.038929,0.040000
75%,0.205000,0.190000,0.230000,0.260000,0.180000,0.160000,0.150000
max,0.930000,0.943881,0.940000,0.976667,0.960000,0.947667,0.925000


In [74]:
#get the day with the highest sum of probability 
day_sum_prob = probabilities_df.sum()
print(day_sum_prob)
df_day_sum_prob = pd.DataFrame(day_sum_prob)
df_day_sum_prob.reset_index()
#renaming the column probabilities 
df_day_sum_prob = df_day_sum_prob.rename(columns={0: 'Cum_prob'})

Friday       250.708309
Monday       241.837160
Saturday     284.448967
Sunday       301.953288
Thursday     225.600195
Tuesday      208.094373
Wednesday    200.357709
dtype: float64


In [75]:
df_day_sum_prob

,Cum_prob
Friday,250.708309
Monday,241.837160
Saturday,284.448967
Sunday,301.953288
Thursday,225.600195
Tuesday,208.094373
Wednesday,200.357709


From the above, we see the day of the week with the highest mean probability is Sunday (0.176272) and Saturday (0.166053). Therefore, the weekends are the days in the week the NCOP should be on stand for as they have the highest probability of crime occurance 

Next we predict the probability for the month 

In [77]:
#label the month 
crime_data['OCC_MONTH'] = label_encoder_trans.fit_transform(crime_data['OCC_MONTH'])
target = crime_data['OCC_MONTH']

In [78]:
#Separate the different numerical,categorica features, target  
categorical_features = ['OCC_DOW','OCC_TIME_RANGE']
numerical_features = ['OCC_DOY','OCC_DAY','OCC_HOUR']
label = ['OCC_MONTH']
target = ['OCC_MONTH']

In [79]:
#Start with day of the week
# split data in train / test
X_train, X_test, y_train, y_test = train_test_split(crime_data[categorical_features + numerical_features]
                                                    ,crime_data[target],
                                                    test_size = 0.3, random_state=1234)

In [80]:
#define transformers and encoder
categorical_transformer = Pipeline(steps=[('onehot',OneHotEncoder(handle_unknown='ignore'))])
numerical_transformer = Pipeline(steps=[('scaler',StandardScaler())])

In [81]:
#building the preprocessor
preprocessor = ColumnTransformer(transformers=[('categorical',categorical_transformer, categorical_features),
                                               ('numerical',numerical_transformer,numerical_features) ])

In [82]:
#define model
random_forest = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestClassifier())])

In [83]:
#fit the model
random_forest.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['OCC_DOW',
                                                   'OCC_TIME_RANGE']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['OCC_DOY', 'OCC_DAY',
                                                   'OCC_HOUR'])])),
                ('regressor', RandomForestClassifier())])

In [84]:
# Predict the probabilities of crime occurrence for each month in the test set
month_probabilities = random_forest.predict_proba(X_test)

In [85]:
# Display the probabilities for each month in the test set
month_names = label_encoder_trans.inverse_transform(random_forest.classes_)
month_probabilities_df = pd.DataFrame(month_probabilities, columns=month_names)
print(month_probabilities_df)

      April  August  December  February  January  July  June  March   May  \
0      0.21    0.03      0.00      0.18     0.08  0.01  0.05   0.42  0.01   
1      0.00    0.94      0.01      0.00     0.02  0.00  0.00   0.00  0.01   
2      0.00    0.00      0.31      0.00     0.00  0.01  0.03   0.01  0.00   
3      0.24    0.08      0.00      0.03     0.09  0.06  0.15   0.00  0.35   
4      0.01    0.10      0.00      0.00     0.01  0.00  0.00   0.00  0.00   
...     ...     ...       ...       ...      ...   ...   ...    ...   ...   
1708   0.00    0.00      0.03      0.00     0.00  0.04  0.00   0.00  0.00   
1709   0.06    0.07      0.02      0.01     0.07  0.06  0.05   0.16  0.42   
1710   0.00    0.05      0.00      0.00     0.00  0.59  0.25   0.00  0.04   
1711   0.07    0.00      0.00      0.63     0.15  0.00  0.07   0.05  0.03   
1712   0.00    0.01      0.00      0.00     0.00  0.99  0.00   0.00  0.00   

      November  October  September  
0         0.01     0.00       0.00  
1

In [86]:
month_predicted_classes = month_probabilities.argmax(axis=1)
# Inverse transform the numerical class labels back to month names
predicted_month_names = label_encoder_trans.inverse_transform(month_predicted_classes)
print(predicted_month_names)
print(month_predicted_classes)

['March' 'August' 'November' ... 'July' 'February' 'July']
[7 1 9 ... 5 3 5]


In [87]:
# Calculate accuracy
month_accuracy = accuracy_score(y_test, month_predicted_classes)
print("Accuracy:", month_accuracy)

Accuracy: 0.9334500875656743


In [88]:
# Calculate confusion matrix
month_conf_matrix = confusion_matrix(y_test, month_predicted_classes)
print("Confusion Matrix:")
print(month_conf_matrix)

Confusion Matrix:
[[124   0   0   2   0   0   1   7   6   0   0   0]
 [  0 180   0   0   0   0   0   0   0   0   0   0]
 [  0   0 118   0   0   0   0   0   0   6   0   0]
 [  5   0   0  63  10   0   0  20   0   0   0   0]
 [  0   0   0   9 140   0   0   6   0   0   0   0]
 [  0   2   0   0   0 176   0   0   0   0   0   0]
 [  0   0   0   0   0   0 133   0   0   0   0   0]
 [ 11   0   0   8   0   0   0  99   1   0   0   0]
 [  5   0   0   0   0   0   0   0 146   0   0   0]
 [  0   0   0   0   0   0   0   0   0 110   7   0]
 [  0   0   0   0   0   0   0   0   0   2 149   2]
 [  0   2   0   0   0   0   0   0   0   0   2 161]]


In [89]:
# Calculate classification report (includes precision, recall, and F1-score for each class)
month_classification_rep = classification_report(y_test, month_predicted_classes, target_names=label_encoder_trans.classes_)
print("Classification Report:")
print(month_classification_rep)

Classification Report:
              precision    recall  f1-score   support

       April       0.86      0.89      0.87       140
      August       0.98      1.00      0.99       180
    December       1.00      0.95      0.98       124
    February       0.77      0.64      0.70        98
     January       0.93      0.90      0.92       155
        July       1.00      0.99      0.99       178
        June       0.99      1.00      1.00       133
       March       0.75      0.83      0.79       119
         May       0.95      0.97      0.96       151
    November       0.93      0.94      0.94       117
     October       0.94      0.97      0.96       153
   September       0.99      0.98      0.98       165

    accuracy                           0.93      1713
   macro avg       0.92      0.92      0.92      1713
weighted avg       0.93      0.93      0.93      1713



In [90]:
# Calculate AUC-ROC (requires one-hot encoded y_test and month_probabilities)
y_test_one_hot = pd.get_dummies(y_test)
month_auc_roc = roc_auc_score(y_test_one_hot, month_probabilities, multi_class='ovr')
print("AUC-ROC:", month_auc_roc)

AUC-ROC: 0.9964908337503408


In [91]:
#get the month with the high mean of probability 
month_probabilities_df.describe(include='all')

,April,August,December,February,January,July,June,March,May,November,October,September
count,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000
mean,0.083257,0.106182,0.065575,0.059241,0.072248,0.101273,0.085201,0.078395,0.090070,0.071261,0.091985,0.095312
std,0.163653,0.230531,0.178553,0.131503,0.183341,0.224387,0.186019,0.157747,0.185931,0.168663,0.190749,0.205967
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.010000,0.020000,0.000000,0.000000,0.000000,0.020000,0.010000,0.010000,0.010000,0.000000,0.010000,0.010000
75%,0.080000,0.060000,0.020000,0.040000,0.030000,0.060000,0.060000,0.070000,0.070000,0.040000,0.070000,0.060000
max,0.950000,1.000000,0.960000,0.910000,0.990000,1.000000,0.990000,0.940000,0.960000,0.980000,0.970000,0.970000


From the above, we see the months of the year with the highest mean probability is August (0.106182) and July (0.101273). Therefore, the months of August and July should expect surgence in the crime occurance

Next we predict the probability for the hour

In [92]:
#hour of the day  
crime_data['OCC_HOUR'] = label_encoder_trans.fit_transform(crime_data['OCC_HOUR'])
target = crime_data['OCC_HOUR']

In [93]:
#Separate the different numerical,categorica features, target  
categorical_features = ['OCC_DOW','OCC_TIME_RANGE','OCC_MONTH']
numerical_features = ['OCC_DOY','OCC_DAY']
label = ['OCC_HOUR']
target = ['OCC_HOUR']

In [94]:
#Start with day of the week
# split data in train / test
X_train, X_test, y_train, y_test = train_test_split(crime_data[categorical_features + numerical_features]
                                                    ,crime_data[target],
                                                    test_size = 0.3, random_state=1234)

In [95]:
#define transformers and encoder
categorical_transformer = Pipeline(steps=[('onehot',OneHotEncoder(handle_unknown='ignore'))])
numerical_transformer = Pipeline(steps=[('scaler',StandardScaler())])

In [96]:
preprocessor = ColumnTransformer(transformers=[('categorical',categorical_transformer, categorical_features),
                                               ('numerical',numerical_transformer,numerical_features) ])

In [97]:
#define model
random_forest = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestClassifier())])

In [98]:
random_forest.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['OCC_DOW', 'OCC_TIME_RANGE',
                                                   'OCC_MONTH']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['OCC_DOY', 'OCC_DAY'])])),
                ('regressor', RandomForestClassifier())])

In [99]:
# Predict the probabilities of crime occurrence for each hour of the day in the test set
hour_probabilities = random_forest.predict_proba(X_test)

In [100]:
# Display the probabilities for each hour of the day in the test set
hour_names = label_encoder_trans.inverse_transform(random_forest.classes_)
hour_probabilities_df = pd.DataFrame(hour_probabilities, columns=hour_names)
print(hour_probabilities_df)

          0         1     2      3         4         5     6     7     8   \
0     0.0000  0.000000  0.00  0.000  0.000000  0.000000  0.08  0.07  0.16   
1     0.0000  0.000000  0.00  0.000  0.000000  0.000000  0.00  0.00  0.00   
2     0.0000  0.000000  0.00  0.000  0.000000  0.000000  0.00  0.00  0.00   
3     0.0825  0.328000  0.01  0.562  0.000000  0.017500  0.00  0.00  0.00   
4     0.2400  0.046667  0.15  0.500  0.063333  0.000000  0.00  0.00  0.00   
...      ...       ...   ...    ...       ...       ...   ...   ...   ...   
1708  0.0000  0.000000  0.00  0.000  0.000000  0.000000  0.00  0.00  0.00   
1709  0.0000  0.000000  0.00  0.000  0.000000  0.000000  0.00  0.00  0.00   
1710  0.0100  0.000000  0.00  0.000  0.000000  0.000000  0.00  0.00  0.00   
1711  0.3620  0.008000  0.54  0.000  0.090000  0.000000  0.00  0.00  0.00   
1712  0.0100  0.010000  0.01  0.940  0.011667  0.018333  0.00  0.00  0.00   

        9   ...        14        15        16        17    18     19     20

In [101]:
hour_predicted_classes = hour_probabilities.argmax(axis=1)
# Inverse transform the numerical class labels back to hour names
predicted_hour_names = label_encoder_trans.inverse_transform(hour_predicted_classes)
print(predicted_hour_names)
print(hour_predicted_classes)

[ 9 12 18 ... 15  2  3]
[ 9 12 18 ... 15  2  3]


In [102]:
# Calculate accuracy
hour_accuracy = accuracy_score(y_test, hour_predicted_classes)
print("Accuracy:", hour_accuracy)

Accuracy: 0.19147694103911267


In [103]:
# Calculate confusion matrix
hour_conf_matrix = confusion_matrix(y_test, hour_predicted_classes)
print("Confusion Matrix:")
print(hour_conf_matrix)

Confusion Matrix:
[[73 32 39 28 18 15  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [28 16 41 15 10  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [36 17 27 22 10  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [40  9 17 15 12  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [16  9 13 16  4  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [14  3  9  6  5  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  6  1  0  3  6  4  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  2  4  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  3  3  5  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  3  3  2  6  2  2  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  4  4  1  2  2  7  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  2 10  0  6  7  6  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  3  8  1  1  8  8  0  0  0  0  0  0]
 [ 0  0

In [104]:
# Calculate AUC-ROC (requires one-hot encoded y_test and hour_probabilities)
y_test_one_hot = pd.get_dummies(y_test)
hour_auc_roc = roc_auc_score(y_test_one_hot, hour_probabilities, multi_class='ovr')
print("AUC-ROC:", hour_auc_roc)

AUC-ROC: 0.8530609620639732


In [105]:
#get the day with the high mean of probability 
hour_probabilities_df.describe(include='all')

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,...,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000
mean,0.117828,0.053952,0.080239,0.061120,0.036020,0.021571,0.009832,0.010742,0.006565,0.012395,...,0.023256,0.026251,0.024803,0.039468,0.040252,0.059747,0.058527,0.078282,0.082598,0.090266
std,0.227454,0.143960,0.180951,0.151394,0.118491,0.090980,0.056034,0.061630,0.040490,0.066865,...,0.091331,0.099612,0.096971,0.132195,0.121732,0.148286,0.149198,0.176606,0.178432,0.194447
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.106000,0.020000,0.040000,0.020333,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.010000,0.030000,0.024286,0.046667,0.058333,0.050000
max,0.940000,0.973333,0.950000,0.940000,0.910000,0.810000,0.865000,0.750000,0.650000,0.730000,...,0.790000,0.860000,0.870000,0.820000,0.895000,0.940000,0.884000,0.900000,0.960000,0.910000


From the above, we see the hour of the day with the highest mean probability is midnight - 00:00 (0.117828) and 23:00 (0.090266). Therefore, the hours of 23:00 to midnight should expect surgence in the crime occurance

From our model we can say that during the weekends (saturday and sunday) of July and August in the time of 23:00 to midnight, the NCOP should expect occurance of crimes and should dedicate more planning towards this time period